In [63]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import env
import wrangle as w

from sqlalchemy import text, create_engine

from datetime import date

import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)


In [89]:
def read_sql_query(query, db):
    """
    This function will 
    - accept two strings: an sql query, and the database name
    - read the query from the database into a dataframe
    - return the dataframe
    """
    # using "new" (May 2023) version of reading sql queries with pandas

    # define the database url
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{db}'
    # create the connection
    engine = create_engine(url)
    connection = engine.connect()
    # create the query using text() and the string that has the sql query
    query_t = text(query)

    df = pd.read_sql(query_t, connection)

    return df

In [94]:
def wrangle_curriculum_logs():
    """
    This function will 
    - read in curriculum data from a txt file ('anonymized-curriculum-access.txt')
    - separate it into columns
    - set date column as the index (as a datetime) and sort the index
    - fill null values in cohort_id with 0
    - then read in data from a sql query to get cohort names and dates
    - merge the two dataframes together on cohort_id
    - return the merged df
    """
    colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
    df = pd.read_csv("anonymized-curriculum-access.txt", 
                     sep="\s", 
                     header=None, 
                     names = colnames, 
                     usecols=[0, 2, 3, 4, 5])
    # fill null values with 0 and make this float column into integer column
    df.cohort_id = df.cohort_id.fillna(0)
    df.cohort_id = df.cohort_id.astype('int64')
    
    # pull second dataframe from mysql database
    # define query and database to pull from 
    query = 'SELECT id, name, start_date, end_date, program_id FROM cohorts'
    db = 'curriculum_logs'
    # read in the sql query 
    cohort_df = read_sql_query(query, db)
    # add a cohort id of 0 to account for the null values in the previous df
    cohort_df.loc[len(cohort_df.index)] = [0,'Unknown cohort', '2000-01-01', '2000-01-01', 0]
    # merge the dataframes together
    new_df = pd.merge(df, cohort_df, left_on='cohort_id', right_on='id')
    # drop the repeated column
    new_df = new_df.drop(columns=['id'])
    
    # setting date column as the index
    new_df.date = new_df.date.astype('datetime64')
    new_df = new_df.set_index('date')
    new_df = new_df.sort_index()
    
    return new_df
    

In [95]:
df = wrangle_curriculum_logs()

In [96]:
df.head()

,endpoint,user_id,cohort_id,source_ip,name,start_date,end_date,program_id
date,,,,,,,,
2018-01-26,/,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
2018-01-26,javascript-i/introduction/working-with-data-ty...,39,22,97.105.19.61,Teddy,2018-01-08,2018-05-17,2
2018-01-26,javascript-i/javascript-with-html,39,22,97.105.19.61,Teddy,2018-01-08,2018-05-17,2
2018-01-26,javascript-i/javascript-with-html,2,22,97.105.19.61,Teddy,2018-01-08,2018-05-17,2
2018-01-26,javascript-i/functions,2,22,97.105.19.61,Teddy,2018-01-08,2018-05-17,2


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   endpoint    900222 non-null  object
 1   user_id     900223 non-null  int64 
 2   cohort_id   900223 non-null  int64 
 3   source_ip   900223 non-null  object
 4   name        900223 non-null  object
 5   start_date  900223 non-null  object
 6   end_date    900223 non-null  object
 7   program_id  900223 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 61.8+ MB


In [45]:
df.cohort_id = df.cohort_id.fillna(0)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   endpoint   900222 non-null  object 
 1   user_id    900223 non-null  int64  
 2   cohort_id  900223 non-null  float64
 3   source_ip  900223 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 34.3+ MB


In [81]:
query = 'SELECT id, name, start_date, end_date, program_id FROM cohorts'
db = 'curriculum_logs'

In [82]:
cohort_df = read_sql_query(query, db)

In [83]:
cohort_df.tail()

,id,name,start_date,end_date,program_id
48,135,Marco,2021-01-25,2021-07-19,2
49,136,Placeholder for students in transition,2021-03-03,2029-03-01,2
50,137,Florence,2021-03-15,2021-09-03,3
51,138,Neptune,2021-03-15,2021-09-03,2
52,139,Oberon,2021-04-12,2021-10-01,2


In [84]:
cohort_df.loc[len(cohort_df.index)] = [0,'Unknown cohort', '2000-01-01', '2000-01-01', 0]

In [85]:
cohort_df.tail()

,id,name,start_date,end_date,program_id
49,136,Placeholder for students in transition,2021-03-03,2029-03-01,2
50,137,Florence,2021-03-15,2021-09-03,3
51,138,Neptune,2021-03-15,2021-09-03,2
52,139,Oberon,2021-04-12,2021-10-01,2
53,0,Unknown cohort,2000-01-01,2000-01-01,0


In [86]:
cohort_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54 non-null     int64 
 1   name        54 non-null     object
 2   start_date  54 non-null     object
 3   end_date    54 non-null     object
 4   program_id  54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.5+ KB


In [87]:
new_df = pd.merge(df, cohort_df, left_on='cohort_id', right_on='id')

In [88]:
new_df.head()

,endpoint,user_id,cohort_id,source_ip,id,name,start_date,end_date,program_id
0,/,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
1,java-ii,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
2,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
3,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
4,javascript-i/functions,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
